In [1]:
import os
import session_info
import pandas as pd
import numpy as np
import tensorflow as tf
from random import sample

In [2]:
session_info.show()

In [3]:
if os.path.isdir('infer') == False :
    os.mkdir("infer")

checkColumns = ['loading',
                'measurement_0',
                'measurement_1',
                'measurement_2',
                'measurement_3',
                'measurement_4',
                'measurement_5',
                'measurement_6',
                'measurement_7',
                'measurement_8',
                'measurement_9',
                'measurement_10',
                'measurement_11',
                'measurement_12',
                'measurement_13',
                'measurement_14',
                'measurement_15',
                'measurement_16',
                'miss3',
                'miss4',
                'miss5'
               ]


## Preprocessing with test data

In [4]:
def outp(name) :
    test = pd.read_csv('test.csv')
    pd_id = test['id']

    test['miss3'] = test['measurement_3'].isna().astype(int)*32
    test['miss4'] = test['measurement_4'].isna().astype(int)*32
    test['miss5'] = test['measurement_5'].isna().astype(int)*32
    test['miss6'] = test['measurement_6'].isna().astype(int)*32

    for i in checkColumns :
        m = test[i].mean()
        test[i] = test[i].fillna(m)
    test = test[checkColumns]

    predictions = model.predict(test)
    predictions = pd.DataFrame(predictions, columns=['failure'])
    out = pd.concat([pd_id, predictions],axis=1)
    out.to_csv(f'infer/{name}.csv',index=False)

## Loading models with top 5 private score

In [5]:
for h5 in os.listdir('goodmodel'):
    model = tf.keras.models.load_model('goodmodel/'+h5)
    outp(h5.replace(".h5","").replace("0.",""))

650/650 [==============================] - 1s 1ms/step


### Model Assembling

- Choose models that got top-5 score in the Kaggle private leaderboard and re-made the prediction.

- In the end, I chose the max ranking value to do the prediction in those prediction data after several experiments.

In [6]:
pre = []
test = pd.read_csv('test.csv')
pd_id = test['id']
dirlist = os.listdir('infer')
for doc in dirlist:
    df = pd.read_csv('infer/'+doc)
    # Get the value after ranking with the failure rate
    rank = df['failure'].rank(axis=0).rename(doc)
    pd_id = pd.concat([pd_id, rank],axis=1)

pd_id['min'] = pd_id.drop('id', axis=1).min(axis=1)
pd_id['max'] = pd_id.drop('id', axis=1).max(axis=1)
pd_id['mean'] = pd_id.drop('id', axis=1).mean(axis=1)
pd_id['median'] = pd_id.drop('id', axis=1).median(axis=1)

df = pd_id['max'].copy()
# Normalization
normalized_df=(df-df.min())/(df.max()-df.min())
out = pd.concat([test['id'], normalized_df.rename('failure')],axis=1)

# Generate prediction data
out.to_csv('inference.csv',index=False)
pd_id

,id,model_5847.csv,model_5856.csv,model_5859.csv,model_5869.csv,model_587.csv,model_5872.csv,model_5875.csv,min,max,mean,median
0,26570,9771.0,10200.0,9965.0,10108.0,9888.0,10051.0,10080.0,9771.0,10200.0,10003.777778,10027.388889
1,26571,6448.0,7112.0,6346.0,6425.0,7290.0,6438.0,6714.0,6346.0,7290.0,6712.111111,6580.055556
2,26572,6928.0,6926.0,6444.0,7063.0,6750.0,6765.0,6051.0,6051.0,7063.0,6671.222222,6757.500000
3,26573,9539.0,9117.0,9494.0,9082.0,8977.0,9632.0,9979.0,8977.0,9979.0,9419.555556,9456.777778
4,26574,20248.0,19939.0,19983.0,19941.0,20320.5,20005.0,20008.0,19939.0,20320.5,20078.222222,20006.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
20770,47340,14876.0,14977.0,15070.0,15003.0,14360.0,15148.0,13544.0,13544.0,15148.0,14630.000000,14926.500000
20771,47341,1402.0,1239.0,1582.0,1240.0,1292.0,1352.0,1509.0,1239.0,1582.0,1381.888889,1366.944444
20772,47342,3227.0,1674.0,4167.0,3181.0,1298.0,917.0,1436.0,917.0,4167.0,2331.555556,2002.777778
20773,47343,11604.0,11322.0,11379.0,11835.0,11405.0,11882.0,12043.0,11322.0,12043.0,11648.333333,11626.166667
